In [1]:
path = '../blockchain_files/3000000to3999999_BlockTransaction/'

In [19]:
import pandas as pd
from datetime import datetime
import time
import numpy as np
import torch
from torch_geometric.data import Data
import torch.nn as nn
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
from torch.nn.utils.rnn import pack_sequence

In [3]:
df = pd.read_csv(path+'final.csv')

In [4]:
labels = df['0'].unique()
for label in labels:
    print('%s: '% label,(df['0'] == label).sum())
# print('mining: ',(merged_df['0'] == 'mining').sum())

mining:  3499995
nan:  0
exchange:  1315278
walletapp:  52765
compromised:  24937
gambling:  823
isowallet:  135
phish_hack:  172


In [5]:
print(labels[2])
exchange_data = df[df['0'] == labels[2]]
exchange_data.head()

exchange


,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
22,1484475182,13,1676,0.989400,33.3333,3.0,2.1,exchange
26,1484475182,13,3522,2500.000000,33.3333,3.0,2.1,exchange
37,1484475230,17,54938,25.237936,4.0000,6.0,2.1,exchange
80,1484475418,33,1890243,0.010000,9.0000,2.0,2.1,exchange
84,1484475449,17,55130,0.322861,4.0000,6.0,2.1,exchange


In [6]:
exchange_addrs = exchange_data['from'].unique()

In [7]:
kept_ex_addrs = []
for addr in exchange_addrs:
    if (exchange_data['from']==addr).sum() > 100:
        kept_ex_addrs.append(addr)
#     print( str(addr)+' :',(exchange_data['from']==addr).sum())
    

In [8]:
print(labels[0])
mining_data = df[df['0'] == labels[0]]
mining_data.head()

mining


,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4.950000,3.9,2.0,2.1,mining
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining


In [9]:
mining_addrs = mining_data['from'].unique()

In [10]:
kept_mining_addrs = []
for addr in mining_addrs:
    if (mining_data['from']==addr).sum() > 100:
        kept_mining_addrs.append(addr)
#     print( str(addr)+' :',(mining_data['from']==addr).sum())
    

In [11]:
def receiving_transaction_graph_h4_timed(node,st_time,end_time,dataframe):
    
    timed_df = dataframe[dataframe['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['to'] == node]
    
    neigh2 = timed_df[timed_df['to'].isin(neigh1['from'])]
    neigh2=neigh2[neigh2['to'] != node]
    
    neigh3 = timed_df[timed_df['to'].isin(neigh2['from'])]
    neigh3 = neigh3[~neigh3['to'].isin(neigh2['to'])]
    
    neigh4 = timed_df[timed_df['to'].isin(neigh3['from'])]
    neigh4 = neigh4[~neigh4['to'].isin(neigh3['to'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    

In [12]:
def send_extract_graph_h4_timed(node,st_time,end_time,dataframe):
    
    timed_df = dataframe[dataframe['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['from'] == node]
    
    neigh2 = timed_df[timed_df['from'].isin(neigh1['to'])]
    neigh2=neigh2[neigh2['from'] != node]
    
    neigh3 = timed_df[timed_df['from'].isin(neigh2['to'])]
    neigh3 = neigh3[~neigh3['from'].isin(neigh2['from'])]
    
    neigh4 = timed_df[timed_df['from'].isin(neigh3['to'])]
    neigh4 = neigh4[~neigh4['from'].isin(neigh3['from'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    


In [13]:
def combine(send_graph,rec_graph):
    
    (neigh1_send,neigh2_send,neigh3_send,neigh4_send) = send_graph
    (neigh1_rec,neigh2_rec,neigh3_rec,neigh4_rec) = rec_graph
    
    
    neigh1 = pd.concat([neigh1_send,neigh1_rec])
    neigh2 = pd.concat([neigh2_send,neigh2_rec])
    neigh3 = pd.concat([neigh3_send,neigh3_rec])
    neigh4 = pd.concat([neigh4_send,neigh4_rec])
    
    
    return (neigh1,neigh2,neigh3,neigh4)

st_time = 1484481572
duration = 700000
end_time = st_time+duration
addr = kept_ex_addrs[2]
    
send_graph = send_extract_graph_h4_timed(addr,st_time,end_time,df)
rec_graph = receiving_transaction_graph_h4_timed(addr,st_time,end_time,df)


# (neigh1,neigh2,neigh3,neigh4) = get_aggregated(rec_graph)

(neigh1,neigh2,neigh3,neigh4) = combine(send_graph,rec_graph)
# neigh3

In [14]:
def extract_graphs(addr,duration):
    st_time = df.iloc[0]['timestamp']
#     duration = 700000
#     addr = 261
    end_time = 0
    graphs = []
    while  end_time < df.iloc[-1]['timestamp']:
        end_time = st_time+duration
        
        send_graph = send_extract_graph_h4_timed(addr,st_time,end_time,df)
        rec_graph = receiving_transaction_graph_h4_timed(addr,st_time,end_time,df)
        
        (neigh1,neigh2,neigh3,neigh4) = combine(send_graph,rec_graph)
        graphs.append((neigh1,neigh2,neigh3,neigh4))
        st_time = end_time

        
    return graphs

In [15]:
miner_graph = []
duration = 700000
for addr in kept_mining_addrs:
    miner_graph.append(extract_graphs(addr,duration))

In [16]:
ex_graph = []
duration = 700000
for addr in kept_ex_addrs:
    ex_graph.append(extract_graphs(addr,duration))

In [17]:
ex_graph[0][0][1]

,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
8,1484475096,5,1633800,0.074210,94.0,2.0,2.1,NaN
16,1484475156,5,12,0.054131,94.0,2.0,2.1,NaN
37,1484475230,17,54938,25.237936,4.0,6.0,2.1,exchange
59,1484475316,28,8499,0.270721,2.5,2.0,2.1,NaN
62,1484475316,28,8560,0.151508,2.5,2.0,2.1,NaN
...,...,...,...,...,...,...,...,...
229680,1485175025,28,6895,1.040918,2.5,2.0,2.1,NaN
229682,1485175025,28,8560,0.135487,2.5,2.0,2.1,NaN
229684,1485175025,28,13241,0.015695,2.5,2.0,2.1,NaN
229686,1485175025,28,78986,0.549578,2.5,2.0,2.1,NaN


In [18]:
def get_graph_data(graph,label):
    (neigh1,neigh2,neigh3,neigh4) = graph
    neigh1_nodes = pd.concat([neigh1['from'],neigh1['to']])
    neigh2_nodes = pd.concat([neigh2['from'],neigh2['to']])
    neigh3_nodes = pd.concat([neigh3['from'],neigh3['to']])
    neigh4_nodes = pd.concat([neigh4['from'],neigh4['to']])

    nodes = pd.concat([neigh1_nodes
                  ,neigh2_nodes
                  ,neigh3_nodes
                  ,neigh4_nodes]).unique()
    map_id = {j:i for i,j in enumerate(nodes)}
    
    neigh1['from'] = neigh1['from'].map(map_id)
    neigh1['to'] = neigh1['to'].map(map_id)
    
    neigh2['from'] = neigh2['from'].map(map_id)
    neigh2['to'] = neigh2['to'].map(map_id)
    
    neigh3['from'] = neigh3['from'].map(map_id)
    neigh3['to'] = neigh3['to'].map(map_id)
    
    neigh4['from'] = neigh4['from'].map(map_id)
    neigh4['to'] = neigh4['to'].map(map_id)
    
    graph = pd.concat([neigh1,neigh2,neigh3,neigh4])
    
#     edge_index = torch.tensor(graph[['from','to']].to_numpy().T, dtype=torch.long)
#     print(edge_index.shape)
    
    graph = graph.sort_values(['from','to','timestamp']).reset_index(drop=True)



    def get_groups(graph):
        g = graph.groupby(['from','to'])

        groups = g.groups

        keys = list(groups.keys())
        return keys,groups
    
    def comparison(item):

        return len(item[1])
        
    
    keys,groups = get_groups(graph)
    
    transac_ls = []
    for i in range(len(keys)):
        edge = graph.iloc[groups[keys[i]]][['from','to']].iloc[0].to_numpy()
        tensor = torch.tensor(graph.iloc[groups[keys[i]]][['value','gasLimit','gasPrice','gasUsed']].to_numpy())
        transac_ls.append((edge,tensor))
        
#         print(edge)
#         print(tensor)
        
#         print(graph.iloc[groups[keys[i]]])
        
#         return
        
    lst = sorted(transac_ls, key=comparison,reverse=True)
    edges  = [item[0] for item in lst]
    edge_attributes  = [item[1] for item in lst]
    
    if len(edge_attributes) == 0:
        return edges,edge_attributes,label
    
    edge_attributes = pack_sequence(edge_attributes)
    
    
    return edges,edge_attributes,label
#     n = 0
#     print(graph.iloc[groups[keys[n]]])
        

    
data = get_graph_data(ex_graph[0][0],0)
edges,edge_attributes,label = data


NameError: name 'pack_sequence' is not defined

In [59]:
def get_graph_data(graph,label):
    (neigh1,neigh2,neigh3,neigh4) = graph
    neigh1_nodes = pd.concat([neigh1['from'],neigh1['to']])
    neigh2_nodes = pd.concat([neigh2['from'],neigh2['to']])
    neigh3_nodes = pd.concat([neigh3['from'],neigh3['to']])
    neigh4_nodes = pd.concat([neigh4['from'],neigh4['to']])

    nodes = pd.concat([neigh1_nodes
                  ,neigh2_nodes
                  ,neigh3_nodes
                  ,neigh4_nodes]).unique()
    map_id = {j:i for i,j in enumerate(nodes)}
    
    neigh1['from'] = neigh1['from'].map(map_id)
    neigh1['to'] = neigh1['to'].map(map_id)
    
    neigh2['from'] = neigh2['from'].map(map_id)
    neigh2['to'] = neigh2['to'].map(map_id)
    
    neigh3['from'] = neigh3['from'].map(map_id)
    neigh3['to'] = neigh3['to'].map(map_id)
    
    neigh4['from'] = neigh4['from'].map(map_id)
    neigh4['to'] = neigh4['to'].map(map_id)
    
    graph = pd.concat([neigh1,neigh2,neigh3,neigh4])
    
#     edge_index = torch.tensor(graph[['from','to']].to_numpy().T, dtype=torch.long)
#     print(edge_index.shape)
    
    graph = graph.sort_values(['from','to','timestamp']).reset_index(drop=True)



    def get_groups(graph):
        g = graph.groupby(['from','to'])

        groups = g.groups

        keys = list(groups.keys())
        return keys,groups
    
    def comparison(item):

        return len(item[1])
        
    
    keys,groups = get_groups(graph)
    
    transac_ls = []
    for i in range(len(keys)):
        edge = graph.iloc[groups[keys[i]]][['from','to']].iloc[0].to_numpy()
        tensor = torch.tensor(graph.iloc[groups[keys[i]]][['value','gasLimit','gasPrice','gasUsed']].to_numpy())
        transac_ls.append((edge,tensor.float()))
        
#         print(edge)
#         print(tensor)
        
#         print(graph.iloc[groups[keys[i]]])
        
#         return
        
    lst = sorted(transac_ls, key=comparison,reverse=True)
    edges  = [item[0] for item in lst]
    edge_attributes  = [item[1] for item in lst]
    edge_attrib_lenghts  = [len(item[1]) for item in lst]
    
    if len(edge_attributes) == 0:
        return edges,edge_attributes,edge_attrib_lenghts,label
    
#     print(edge_attributes[:5])
    
    edge_attributes = pack_sequence(edge_attributes)
    
    
    return edges,edge_attributes,edge_attrib_lenghts,label
#     n = 0
#     print(graph.iloc[groups[keys[n]]])
        

    
data = get_graph_data(ex_graph[0][0],0)
edges,edge_attributes,edge_attrib_lenghts,label = data


In [60]:
ins = unpack_sequence(edge_attributes,edge_attrib_lenghts)
# ins[:5]

In [46]:
from torch.nn.utils.rnn import pack_sequence

In [47]:
rnn = nn.RNN(4, 16, 1)
inputs = torch.randn(5, 3, 4)
h0 = torch.randn(1, 3, 16)
output, hn = rnn(inputs, h0)
print(output.shape)
print(hn.shape)

torch.Size([5, 3, 16])
torch.Size([1, 3, 16])


In [48]:
output, hn = rnn(edge_attributes)

In [49]:
hn.shape

torch.Size([1, 16584, 16])

In [50]:
out = unpack_sequence(output,edge_attrib_lenghts)
out[0].shape

torch.Size([731, 16])

In [39]:
ins = unpack_sequence(edge_attributes,edge_attrib_lenghts)
ins[0].shape

torch.Size([731, 4])

torch.Size([731, 16])

In [245]:
edges = [item[['from','to']].iloc[0].to_numpy() for item in lst1] 

In [208]:
out_unpacked = unpack_sequence(output,lengths)
# hn_unpacked = unpack_sequence(hn,lengths)

In [42]:
data_list = []
def add_graphs_to_list(graphs,label):
    for addr_graphs in graphs:
        for graph in addr_graphs:
            data_list.append(get_graph_data(graph,label))
            

add_graphs_to_list(ex_graph,0)
add_graphs_to_list(miner_graph,1)

In [43]:
import pickle


In [44]:
with open(path+"graphs_data_rnn_1", "wb") as fp:   #Pickling
    pickle.dump(data_list, fp)

In [29]:
with open(path+"graphs_data_rnn", "rb") as fp:   # Unpickling
    read_graph_data = pickle.load(fp)

In [30]:
((read_graph_data[0][0].edge_index == data_list[0][0].edge_index)==False).sum()

tensor(0)

In [29]:
import torch.nn.utils.rnn as rnnutils

def unpack_sequence(packed_sequence, lengths):
    assert isinstance(packed_sequence, rnnutils.PackedSequence)
    head = 0
    trailing_dims = packed_sequence.data.shape[1:]
    unpacked_sequence = [torch.zeros(l, *trailing_dims) for l in lengths]
    # l_idx - goes from 0 - maxLen-1
    for l_idx, b_size in enumerate(packed_sequence.batch_sizes):
        for b_idx in range(b_size):
            unpacked_sequence[b_idx][l_idx] = packed_sequence.data[head]
            head += 1
    return unpacked_sequence